In [1]:
from tqdm.notebook import tqdm
import random
import torch
from torch import nn
from torch import optim
import numpy as np

In [2]:
data = np.load("eu_d.npy", allow_pickle=True)
sq = data ** 2
prod = (data[:, 0] * data[:, 1])[:, None]
uod = (data[:, 0] / data[:, 1])[:, None]
u2od2 = uod ** 2
inv_d2 = sq[:, 1][:, None]
data = np.concatenate([data, sq, prod, inv_d2, uod, u2od2], axis=1)
np.random.shuffle(data)

In [3]:
data.shape

(3116150, 8)

In [4]:
class ResBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LeakyReLU(),
            nn.BatchNorm1d(dim),
        )
        
    def forward(self, x):
        return x + self.net(x)

In [12]:
net = nn.Sequential(
    nn.Linear(8,16),
    nn.LeakyReLU(),
    nn.BatchNorm1d(16),
    ResBlock(16),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
    ResBlock(16),
    nn.Linear(16,1),
    nn.Softplus(),
)

In [13]:
lr = 1e-2
gamma = 0.95

In [14]:
optimizer = optim.AdamW(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
criterion = nn.MSELoss()

In [15]:
train_loader = torch.utils.data.DataLoader(data[:-100000], batch_size=10000)

In [16]:
val_loader = torch.utils.data.DataLoader(data[-100000:], batch_size=10000)

In [17]:
bar_info = {}

In [19]:
for epoch in range(1000):
    bar_info.update({"lr": scheduler.get_last_lr()[0]})
    net.train()
    for u_d in (bar := tqdm(train_loader)):
#         eu_d = torch.rand_like(eu_d) * 10
        net.zero_grad()
        sqdiff = torch.tensor((u_d[:, 0] - u_d[:, 1]) / u_d[:, 1])[:, None].square()
        pred = net(u_d)
        loss = criterion(pred, sqdiff)
        loss.backward()
        optimizer.step()
        bar_info.update({"err": np.sqrt(loss.item())})
        bar.set_postfix(bar_info)
    scheduler.step()
    net.eval()
    err = []
    err_ratio = []
    with torch.no_grad():
        for u_d in val_loader:
#             eu_d = torch.rand_like(eu_d) * 10
            sqdiff = torch.tensor((u_d[:, 0] - u_d[:, 1]) / u_d[:, 1])[:, None].square()
            pred = net(u_d)
            err += torch.abs(pred - sqdiff).tolist()
            err_ratio += torch.abs((pred - sqdiff) / sqdiff).tolist()
        print(torch.cat([sqdiff, pred], dim=1)[:10, :])
    print(epoch, np.mean(err), np.median(err_ratio), np.mean(err_ratio))

  0%|          | 0/302 [00:00<?, ?it/s]

<ipython-input-19-79099b937306>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sqdiff = torch.tensor((u_d[:, 0] - u_d[:, 1]) / u_d[:, 1])[:, None].square()
<ipython-input-19-79099b937306>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sqdiff = torch.tensor((u_d[:, 0] - u_d[:, 1]) / u_d[:, 1])[:, None].square()


tensor([[0.0933, 0.0929],
        [0.1077, 0.1065],
        [0.0633, 0.0622],
        [0.0045, 0.0043],
        [0.0238, 0.0236],
        [0.1338, 0.1315],
        [0.0869, 0.0846],
        [0.0073, 0.0077],
        [0.1112, 0.1097],
        [0.0006, 0.0022]])
0 0.0012997072551349993 0.030516049824655056 5489.990356876527


  0%|          | 0/302 [00:00<?, ?it/s]

tensor([[0.0933, 0.0930],
        [0.1077, 0.1064],
        [0.0633, 0.0622],
        [0.0045, 0.0043],
        [0.0238, 0.0235],
        [0.1338, 0.1314],
        [0.0869, 0.0846],
        [0.0073, 0.0077],
        [0.1112, 0.1096],
        [0.0006, 0.0022]])
1 0.0013044953651091783 0.03011482860893011 5473.580261356086


...

tensor([[0.0933, 0.0935],
        [0.1077, 0.1066],
        [0.0633, 0.0632],
        [0.0045, 0.0043],
        [0.0238, 0.0233],
        [0.1338, 0.1324],
        [0.0869, 0.0859],
        [0.0073, 0.0075],
        [0.1112, 0.1100],
        [0.0006, 0.0021]])
76 0.0011429750137028167 0.029630791395902634 4785.113380429029


  0%|          | 0/302 [00:00<?, ?it/s]

tensor([[0.0933, 0.0935],
        [0.1077, 0.1066],
        [0.0633, 0.0632],
        [0.0045, 0.0043],
        [0.0238, 0.0233],
        [0.1338, 0.1324],
        [0.0869, 0.0859],
        [0.0073, 0.0075],
        [0.1112, 0.1100],
        [0.0006, 0.0021]])
77 0.0011426699579608977 0.02962195035070181 4784.297893704185


  0%|          | 0/302 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [192]:
data[-100000:][np.argmax(err)]

array([2.0000076, 2.       , 4.0000305, 4.       , 4.0000153],
      dtype=float32)

In [439]:
print(net(torch.tensor([[0., 0.],
                        [0., 1],
                        [0., 2],
                        [0., 3],
                        [0., 4],
                        [0., 5],
                        [0., 6],
                        [0., 7],
                        [0., 8],
                        [0., 9]])))

tensor([[ 0.8370],
        [ 1.3064],
        [ 3.7761],
        [ 8.9825],
        [17.3631],
        [27.4014],
        [37.5714],
        [47.5369],
        [57.0769],
        [66.3386]], grad_fn=<AddmmBackward>)


In [112]:
net.eval()
err = []
with torch.no_grad():
    for eu_d in (bar := tqdm(data[-5000:, :])):
        sqdiff = torch.tensor(eu_d[0] - eu_d[1]).square()
        pred = net(torch.tensor(eu_d))
        err.append(np.abs(pred.item() - sqdiff.item()) / sqdiff.item())
        print(pred.item(), sqdiff.item())

  0%|          | 0/10000 [00:00<?, ?it/s]

0.4058016538619995 1.343533992767334
24.172975540161133 28.480993270874023
3.702401876449585 3.4859619140625
0.46427279710769653 0.11884527653455734
0.4227063059806824 1.0068390369415283
0.46118849515914917 0.008778247982263565
0.4758131504058838 0.03400048986077309
0.4633966088294983 0.10225093364715576
0.46653103828430176 0.006437885574996471
0.41779059171676636 0.6680024266242981
0.4586895704269409 0.09229059517383575
0.4427664875984192 0.5741840600967407
0.4328789710998535 0.302799791097641
0.6527474522590637 0.44908860325813293
0.42768287658691406 0.7422394156455994
0.4410039782524109 0.10152947157621384
0.4796168804168701 0.016825322061777115
0.465549111366272 0.2962101399898529
0.4572637677192688 0.03287649154663086
0.47245150804519653 0.0564974844455719
10.757575035095215 6.9920334815979
0.4306986927986145 0.3750700354576111
0.45801401138305664 0.10852419584989548
0.4778231978416443 0.007147960364818573
0.43711429834365845 0.5711864233016968
0.4567188620567322 0.273633509874343

In [113]:
np.median(err)

0.6662024000679129

In [222]:
net.eval()

Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=4, out_features=8, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=8, out_features=8, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Linear(in_features=8, out_features=8, bias=True)
  (10): LeakyReLU(negative_slope=0.01)
  (11): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=8, out_features=4, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (15): Linear(in_features=4, out_features=2, bias=True)
  (16): LeakyReLU(negative

In [252]:
net(torch.tensor([[0., 0.],[0., .5]]))

tensor([[0.1162],
        [0.1162]], grad_fn=<AddmmBackward>)